In [1]:
import numpy as np
from src.NN import NN
import src.utils as utils

# Settings
csv_filename = "data/seeds_dataset.csv"
hidden_layers = [1,2] # number of nodes in hidden layers i.e. [layer1, layer2, ...]
eta = 0.1 # learning rate
n_epochs = 50 # number of training epochs
n_folds = 4 # number of folds for cross-validation
seed_crossval = 1 # seed for cross-validation
seed_weights = 1 # 

In [2]:

print("Reading '{}'...".format(csv_filename))
X, y, n_classes = utils.read_csv(csv_filename, target_name="y", normalize=True)

Reading 'data/seeds_dataset.csv'...


In [3]:
# m,n=X.shape
# n
config = {
        n_classes : 25,
        "hidden_layers" : [1,2], # number of nodes in hidden layers i.e. [layer1, layer2, ...],
        "eta" : 0.1, # learning rate,
        "n_epochs" : 50, # number of training epochs,
        "n_folds" : 4 ,# number of folds for cross-validation,
        "seed_crossval" : 1 
    }

In [4]:
config.n_clases

AttributeError: 'dict' object has no attribute 'n_clases'

In [22]:
import numpy as np
from src.NN import NN
import src.utils as utils
import copy
import random
from box import Box 
from datetime import datetime
# if(__name__ == 'main')
# Settings

def split_learn(X,y,modelConfig):
    datasets,features=X.shape
    
    splitWeights=list()
    splits = np.hsplit(X,features)
    # return splits
    print("splits made = ",len(splits))
    for o,oX in enumerate(splits):
        N, d = oX.shape

        hidden_layers = modelConfig.hidden_layers # number of nodes in hidden layers i.e. [layer1, layer2, ...]
        eta = modelConfig.eta # learning rate
        n_epochs = modelConfig.n_epochs# number of training epochs
        n_folds = modelConfig.n_folds# number of folds for cross-validation
        seed_crossval =modelConfig.seed  # seed for cross-validation
        n_classes = modelConfig.n_classes


        print("Neural network model:")
        print(" input_dim = {}".format(d))
        print(" hidden_layers = {}".format(hidden_layers))
        print(" output_dim = {}".format(n_classes))
        print(" eta = {}".format(eta))
        print(" n_epochs = {}".format(n_epochs))
        print(" n_folds = {}".format(n_folds))
        print(" seed_crossval = {}".format(seed_crossval))
        # print(" seed_weights = {}\n".format)

        # Create cross-validation folds
        idx_all = np.arange(0, N)
        idx_folds = utils.crossval_folds(N, n_folds, seed=seed_crossval) # list of list of fold indices
        
        # Train/evaluate the model on each fold
        acc_train, acc_valid = list(), list()
        print("Cross-validating with {} folds...".format(len(idx_folds)))
        for i, idx_valid in enumerate(idx_folds):
            #  seed=seed_weights
            # Collect training and test data from folds
            idx_train = np.delete(idx_all, idx_valid)
            X_train, y_train = oX[idx_train], y[idx_train]
            X_valid, y_valid = oX[idx_valid], y[idx_valid]

            # Build neural network classifier model and train
            model = NN(input_dim=d, output_dim=n_classes,
                        hidden_layers=hidden_layers)
            model.train(X_train, y_train, eta=eta, n_epochs=n_epochs)

            # Make predictions for training and test data
            ypred_train = model.predict(X_train)
            ypred_valid = model.predict(X_valid)

            # Compute training/test accuracy score from predicted values
            acc_train.append(100*np.sum(y_train==ypred_train)/len(y_train))
            acc_valid.append(100*np.sum(y_valid==ypred_valid)/len(y_valid))

            # Print cross-validation result
            print(" Fold {}/{}: acc_train = {:.2f}%, acc_valid = {:.2f}% (n_train = {}, n_valid {})".format(i+1,n_folds, acc_train[-1], acc_valid[-1], len(X_train), len(X_valid)))

        # Print results
        print(o," : ")
        print("  -> acc_train_avg = {:.2f}%, acc_valid_avg = {:.2f}%".format(sum(acc_train)/float(len(acc_train)), sum(acc_valid)/float(len(acc_valid))))

        splitWeights.append(model.get_weights())
        
    
    return splitWeights

def get_aggregate_weights(wt):
    new=[]
    
    print("wth",new)
    rep = copy.deepcopy(wt)
    for i,mx in enumerate(rep):
        # m2 = wt[i]
        if i==0:
            print("hitting new")
            new = mx
            # print(new[1])
        else:
            for j,lx in enumerate(mx):
                if j==0 and i!=0:
                    # print(new[j],"88",lx)
                    #  np.concatenate((array_, add_row), axi/s=0)
                    new[j] = [np.append(new[j],lx)]
                    # print(len(new[j]),"*******************")
                else:
                    new[j] = np.add(new[j],lx)
                    print(len(new[j]))

    for k in range(len(new)):
        if not k == 0:
            print(k)
            new[k] = np.divide(new[k],len(wt))
            # print(type(l1))
    print( new )
    return new


def get_aggregate_model(shape,newWeights,modelConfig):
    # for oX in enumerate(splits):
    # print("Reading '{}'...".format(csv_filename))
    # X, y, n_classes = utils.read_csv(csv_filename, target_name="y", normalize=True)
    N, d = shape
    #   N, d = X.shape
    hidden_layers = modelConfig.hidden_layers # number of nodes in hidden layers i.e. [layer1, layer2, ...]
    eta = modelConfig.eta # learning rate
    n_epochs = modelConfig.n_epochs# number of training epochs
    n_folds = modelConfig.n_folds# number of folds for cross-validation
    seed_crossval =modelConfig.seed  # seed for cross-validation
    n_classes = modelConfig.n_classes
    print("Neural network model:")
    print(" input_dim = {}".format(d))
    print(" hidden_layers = {}".format(hidden_layers))
    print(" output_dim = {}".format(n_classes))
    print(" eta = {}".format(eta))
    print(" n_epochs = {}".format(n_epochs))
    print(" n_folds = {}".format(n_folds))
    print(" seed_crossval = {}".format(seed_crossval))

    # list of list of fold indices
    model = NN(input_dim=d, output_dim=n_classes,
                        hidden_layers=hidden_layers,AvgWeights=newWeights)
    # Train/evaluate the model on each fold
    return model


def getGenericNN(X,y,modelConfig):
    
    N, d = X.shape
    hidden_layers = modelConfig.hidden_layers # number of nodes in hidden layers i.e. [layer1, layer2, ...]
    eta = modelConfig.eta # learning rate
    n_epochs = modelConfig.n_epochs# number of training epochs
    n_folds = modelConfig.n_folds# number of folds for cross-validation
    seed_crossval =modelConfig.seed  # seed for cross-validation
    n_classes = modelConfig.n_classes

    print(" -> X.shape = {}, y.shape = {}, n_classes = {}\n".format(X.shape, y.shape, n_classes))
    print("Neural network model:")
    print(" input_dim = {}".format(d))
    print(" hidden_layers = {}".format(hidden_layers))
    print(" output_dim = {}".format(n_classes))
    print(" eta = {}".format(eta))
    print(" n_epochs = {}".format(n_epochs))
    print(" n_folds = {}".format(n_folds))
    print(" seed_crossval = {}".format(seed_crossval))

    # Create cross-validation folds
    idx_all = np.arange(0, N)
    idx_folds = utils.crossval_folds(N, n_folds, seed=seed_crossval) # list of list of fold indices

    # Train/evaluate the model on each fold
    acc_train, acc_valid = list(), list()
    print("Cross-validating with {} folds...".format(len(idx_folds)))
    for i, idx_valid in enumerate(idx_folds):

        # Collect training and test data from folds
        idx_train = np.delete(idx_all, idx_valid)
        X_train, y_train = X[idx_train], y[idx_train]
        X_valid, y_valid = X[idx_valid], y[idx_valid]

        # Build neural network classifier model and train
        model = NN(input_dim=d, output_dim=n_classes,
                hidden_layers=hidden_layers)

        model.train(X_train, y_train, eta=eta, n_epochs=n_epochs)

        # Make predictions for training and test data
        ypred_train = model.predict(X_train)
        ypred_valid = model.predict(X_valid)

        # Compute training/test accuracy score from predicted values
        acc_train.append(100*np.sum(y_train==ypred_train)/len(y_train))
        acc_valid.append(100*np.sum(y_valid==ypred_valid)/len(y_valid))

        # Print cross-validation result
        print(" Fold {}/{}: acc_train = {:.2f}%, acc_valid = {:.2f}% (n_train = {}, n_valid = {})".format(
            i+1, n_folds, acc_train[-1], acc_valid[-1], len(X_train), len(X_valid)))

    # Print results
    print("""***************************** GENERIC MODEL TRAINING RESULTS ********************\n  
    -> acc_train_avg = {:.2f}%, acc_valid_avg = {:.2f}% \n\n""".format(sum(acc_train)/float(len(acc_train)), sum(acc_valid)/float(len(acc_valid))))
    
    return model


# def main():
    # seed for cross-validation
    # csv_filename = "data/tennis.csv"
csv_filename = "data/seeds_dataset.csv"
print("Reading '{}'...".format(csv_filename))
X, y, n_c = utils.read_csv(csv_filename, target_name="y", normalize=True)
random.seed(datetime.now())
config = {
    "n_classes" : n_c,
    "hidden_layers" : [2,2], # number of nodes in hidden layers i.e. [layer1, layer2, ...],
    "eta" : 0.1, # learning rate,
    "n_epochs" : 50, # number of training epochs,
    "n_folds" : 4 ,# number of folds for cross-validation,
    "seed" : random.randrange(1,9)
}

config = Box(config)
N,d = X.shape
fold = utils.crossval_folds(N, 5, seed=config.seed) # list of list of fold indices
valid = random.randrange(0,len(fold))
validSets = fold[valid]
allSets = np.arange(0, N)
trainSets = np.delete(allSets, validSets)

nX, nY = X[trainSets], y[trainSets]
vX, vY = X[validSets], y[validSets]

genericModel = getGenericNN(nX, nY, config)
newModelWeights = split_learn(nX,nY,config)
aggregateWeights = get_aggregate_weights(newModelWeights)
newModel = get_aggregate_model(X.shape,aggregateWeights,config)
print("new weights of the aggreagated model are:", newModel.get_weights())
print ( "\n\n running predictions on validation sets held . . . . .  \n\n")
pred_generic = genericModel.predict(vX)
pred_newModel = newModel.predict(vX)

acc_generic = 100*np.sum(vY==pred_generic)/len(vY)
acc_newModel = 100*np.sum(vY==pred_newModel)/len(vY)

print("""\n*****************************FINAL VALIDATION RESULTS ********************\n  
-> accuracy_genric = {:.2f}%, acc_split_learn = {:.2f}% \n\n""".format(acc_generic,acc_newModel))


Reading 'data/seeds_dataset.csv'...
 -> X.shape = (144, 7), y.shape = (144,), n_classes = 3

Neural network model:
 input_dim = 7
 hidden_layers = [2, 2]
 output_dim = 3
 eta = 0.1
 n_epochs = 50
 n_folds = 4
 seed_crossval = 1
Cross-validating with 4 folds...
 Fold 1/4: acc_train = 74.07%, acc_valid = 75.00% (n_train = 108, n_valid = 36)
 Fold 2/4: acc_train = 68.52%, acc_valid = 55.56% (n_train = 108, n_valid = 36)
 Fold 3/4: acc_train = 63.89%, acc_valid = 72.22% (n_train = 108, n_valid = 36)
 Fold 4/4: acc_train = 60.19%, acc_valid = 38.89% (n_train = 108, n_valid = 36)
***************************** GENERIC MODEL TRAINING RESULTS ********************
  
    -> acc_train_avg = 66.67%, acc_valid_avg = 60.42% 


splits made =  7
Neural network model:
 input_dim = 1
 hidden_layers = [2, 2]
 output_dim = 3
 eta = 0.1
 n_epochs = 50
 n_folds = 4
 seed_crossval = 1
Cross-validating with 4 folds...
 Fold 1/4: acc_train = 79.63%, acc_valid = 83.33% (n_train = 108, n_valid 36)
 Fold 2/4: acc

IndexError: list index out of range

In [ ]:
len(aggregateWeights[0][0])

In [ ]:
print(aggregateWeights[0] )#) len(new[1])
print(newModelWeights[0][0])

In [ ]:
# len(model.get_weights()[0][])
datasets,features=X.shape

In [ ]:
new=[]
import copy
print("wth",new)
rep = copy.deepcopy(wt)
for i,mx in enumerate(rep):
    # m2 = wt[i]
    if i==0:
        print("hitting new")
        new = mx
        print(new[1])
    else:
        for j,lx in enumerate(mx):
            if j==0 and i!=0:
                # print(new[j],"88",lx)
                
                new[j] = [np.append(new[j],lx)]
                print(len(new[j]),"*******************")
            else:
                new[j] = np.add(new[j],lx)
                print(len(new[j]))

for k in range(len(new)):
    if not k == 0:
        print(k)
        new[k] = np.divide(new[k],len(wt))
        # print(type(l1))
print( new )

In [ ]:
new_nn =  NN(input_dim=features, output_dim=n_classes,
                        hidden_layers=hidden_layers,AvgWeights=new)

In [ ]:
new_nn.get_weights()

In [ ]:
nw = [[[2.162206105066135,
   2.511103755489059,
   1.0914315598966,
   1.299168673146784,
   1.9712199964449098,
   -0.6693635968466675,
   1.7271965968317686]],
 [[[0.6449008887629116], [1.2760450946757522]],
  [[0.6449008887629116], [1.2760450946757522]]],
 [[[-0.7172583226205111, -0.862754050663147],
   [-0.635781787018038, 0.01427466090778091],
   [-0.5019770862022626, -0.7865638296664071]],
  [[-0.7172583226205111, -0.862754050663147],
   [-0.635781787018038, 0.01427466090778091],
   [-0.5019770862022626, -0.7865638296664071]],
  [[-0.7172583226205111, -0.862754050663147],
   [-0.635781787018038, 0.01427466090778091],
   [-0.5019770862022626, -0.7865638296664071]]]]

In [ ]:
def dp(a, b):
    # print(a,b,"***^^ \n")
    return sum([float(a_ )* float(b_) for (a_, b_) in zip(a, b)])
y=[2.162206105066135,
   2.511103755489059,
   1.0914315598966,
   1.299168673146784,
   1.9712199964449098,
   -0.6693635968466675,
   1.7271965968317686]

# new[1][1]

In [ ]:
dp(new[1][0],y)

In [ ]:
# for oX in enumerate(splits):
print("Reading '{}'...".format(csv_filename))
X, y, n_classes = utils.read_csv(csv_filename, target_name="y", normalize=True)
N, d = X.shape

print("Neural network model:")
print(" input_dim = {}".format(d))
print(" hidden_layers = {}".format(hidden_layers))
print(" output_dim = {}".format(n_classes))
print(" eta = {}".format(eta))
print(" n_epochs = {}".format(n_epochs))
print(" n_folds = {}".format(n_folds))
print(" seed_crossval = {}".format(seed_crossval))
print(" seed_weights = {}\n".format(seed_weights))

# Create cross-validation folds
idx_all = np.arange(0, N)
idx_folds = utils.crossval_folds(N, n_folds, seed=seed_crossval) # list of list of fold indices

# Train/evaluate the model on each fold
acc_train, acc_valid = list(), list()
print("Cross-validating with {} folds...".format(len(idx_folds)))
for i, idx_valid in enumerate(idx_folds):
#  seed=seed_weights
    # Collect training and test data from folds
    idx_train = np.delete(idx_all, idx_valid)
    X_train, y_train = X[idx_train], y[idx_train]
    X_valid, y_valid = X[idx_valid], y[idx_valid]

    # Build neural network classifier model and train
    # model = NN(input_dim=d, output_dim=n_classes,
    #             hidden_layers=hidden_layers)
    new_nn.train(X_train, y_train, eta=eta, n_epochs=n_epochs)

    # Make predictions for training and test data
    ypred_train = new_nn.predict(X_train)
    ypred_valid = new_nn.predict(X_valid)

    # Compute training/test accuracy score from predicted values
    acc_train.append(100*np.sum(y_train==ypred_train)/len(y_train))
    acc_valid.append(100*np.sum(y_valid==ypred_valid)/len(y_valid))

    # Print cross-validation result
    print(" Fold {}/{}: acc_train = {:.2f}%, acc_valid = {:.2f}% (n_train = {}, n_valid {})".format(i+1,n_folds, acc_train[-1], acc_valid[-1], len(X_train), len(X_valid)))

# Print results
# print(o," : ")
print("  -> acc_train_avg = {:.2f}%, acc_valid_avg = {:.2f}%".format(sum(acc_train)/float(len(acc_train)), sum(acc_valid)/float(len(acc_valid))))

In [ ]:
new_nn.get_weights()

In [ ]:
new=[]
import copy
print("wth",new)
rep = copy.deepcopy(wt)
for i,mx in enumerate(rep):
    # m2 = wt[i]
    if i==0:
        print("hitting new")
        new = mx
        print(new[1])
    else:
        for j,lx in enumerate(mx):
            if j==0 and i!=0:
                # print(new[j],"88",lx)
                
                new[j] = np.append(new[j],lx)
                print(len(new[j]),"*******************")
            else:
                new[j] = np.add(new[j],lx)
                print(len(new[j]))

for k in range(len(new)):
    if not k == 0:
        print(k)
        new[k] = np.divide(new[k],len(wt))
        # print(type(l1))
print( new )

In [ ]:
rp=0
for fp in range(len(wt)):
    print(wt[fp][1])
    rp += wt[fp][1][0][0]
    print(rp)
print(rp/7)

In [ ]:
np.asarray([ 3.9273942 ,  1.9839786 ,  1.7497002 ,  3.1951442 ,  1.8707918 ,
       -0.18531519,  1.9123502 ], dtype="float32")

In [ ]:
print(type(wt[1][1]))

In [ ]:
np.asarray([ 3.9273942 ,  1.9839786 ,  1.7497002 ,  3.1951442 ,  1.8707918 ,
       -0.18531519,  1.9123502 ], dtype="float32").tolist()